# openCV 作業：
* 題目 ：請找一段約一分鐘左右的影片，利用上課時學過的影像處理功能，發揮所有創意和想像，將該影片加工特效，產生一段新影片，怕誤解同學的創意，請將你的亮點盡可能寫在影片中的右下角

* 上傳 ：導師會有告訴同學上傳空間, 或可上傳到Youtube 或自己的雲端空間, 把連結給老師(依導師規範為準)

* 繳交日期 ：課後二週

* 評分 ：亮點愈亮，創意愈多，分數愈高 

* 參考 :<br>
https://www.youtube.com/watch?v=Fszu0L1JtqM

In [2]:
import cv2
import numpy as np
import dlib     
from datetime import datetime as dt

cap = cv2.VideoCapture('../video/iu.mp4')    # 開啟影片檔案

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))        # 取得畫面尺寸
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')             # 使用 XVID 編碼
cap.set(cv2.CAP_PROP_POS_MSEC, 0)               #  從第 0 秒開始撥放

RECT = ((200, 20), (570, 390))
(left, top), (right, bottom) = RECT

def roiarea(frame):                  # 取出 ROI 子畫面
    return frame[top:bottom, left:right]

def replaceroi(frame, roi):             # 將 ROI 區域貼回到原畫面
    frame[top:bottom, left:right] = roi
    return frame

bg = None

ratio = cap.get(cv2.CAP_PROP_FRAME_WIDTH) / cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # cap.get得到相機/視訊檔案的屬性
w = 800;  h = int(w / ratio)

out = cv2.VideoWriter('../video/iu_out.mp4', fourcc, fps, (w, h))

detector = dlib.get_frontal_face_detector()
font = 2;    lt = 16

# print(f'frame_w\t\t: {cap.get(cv2.CAP_PROP_FRAME_WIDTH)}\n'
#       f'frame_fps\t: {cap.get(cv2.CAP_PROP_FPS)}\n'
#       f'frame_count\t: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}')


# 建立 VideoWriter 物件，輸出影片至 output.avi
# out = cv2.VideoWriter('../video/iu_out.avi', fourcc, fps, (w, h))

# output_filename = 'output.avi'
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# fps = 30
# output_size = (w, h)
# out = cv2.VideoWriter(output_filename, fourcc, fps, output_size)




while True:
    ret, frame = cap.read()                          #  read frame : ret: True/ False,  frame:image
    if not ret or cv2.waitKey(1) == 27: break             # wait for 1 ms     
    
    frame = cv2.flip(frame, 1)                       # 0 : 上下左右顛倒,  -1 : 上下顛倒   
    frame = cv2.resize(frame, (w, h))    
    cv2.putText(frame, f'{cap.get(cv2.CAP_PROP_POS_FRAMES):.0f} frames, {cap.get(cv2.CAP_PROP_POS_MSEC):.0f} ms', 
                (50, 250), font, .8, (0,0,255), 2, lt)
 
    fms = cap.get(cv2.CAP_PROP_POS_MSEC)
    if fms > 600 and fms < 1000:
        frame = cv2.flip(frame, -1) 
        cv2.imshow('frame', frame)
        out.write(frame)
    
    elif fms > 3000 and fms < 9500:
        roi = roiarea(frame)                   # 取出子畫面
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)        # BGR to HSV

        frame = replaceroi(frame, roi)         # 將處理完的子畫面貼回到原本畫面中

        cv2.rectangle(frame, RECT[0], RECT[1], (0,0,255), 2)      # 在 ROI 範圍處畫個框
        cv2.imshow('frame', frame)
        out.write(frame)
        
    elif fms > 9600 and fms < 15000:
        roi = roiarea(frame)                   # 取出子畫面
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2XYZ)          # BGR to XYZ

        frame = replaceroi(frame, roi)         # 將處理完的子畫面貼回到原本畫面中

        cv2.rectangle(frame, RECT[0], RECT[1], (0,0,255), 2)      # 在 ROI 範圍處畫個框
        cv2.imshow('frame', frame)
        out.write(frame)
        
    elif fms > 16000 and fms < 18000:
        frame = cv2.flip(frame, 0) 
        cv2.imshow('frame', frame)
        out.write(frame)

    elif fms > 19000 and fms < 25000:
        frame = cv2.Canny(frame, 100, 200)
        cv2.imshow('frame', frame)
        out.write(frame)
    
    elif fms > 27000 and fms < 29000:
        lap = cv2.Laplacian(frame, cv2.CV_64F)
        lap = cv2.convertScaleAbs(lap)
        cv2.imshow('frame', lap)
        out.write(lap)
        
    elif fms > 35000 and fms < 32000:
        sobelX = cv2.Sobel(frame, cv2.CV_64F, 1, 0)
        sobelY = cv2.Sobel(frame, cv2.CV_64F, 0, 1)
        sobelX = cv2.convertScaleAbs(sobelX)
        sobelY = cv2.convertScaleAbs(sobelY)
        sobelXY = cv2.addWeighted(sobelX, 0.5, sobelY, 0.5, 0)
        cv2.imshow('frame', sobelXY)
        out.write(sobelXY)
        
    elif fms > 37000 and fms < 40000:
        d=5
        img_D3 = cv2.GaussianBlur(frame, (d, d), 0.3)
        img_D5 = cv2.GaussianBlur(frame, (d, d), 0.5)    
        img_D5_3 = img_D5 - img_D3   # try img_G0 + img_G1
        cv2.imshow('frame', img_D5_3)
        out.write(img_D5_3)

    elif fms > 40500 and fms < 50000:
            face_rects, scores, idx = detector.run(frame, 0)  # 偵測人臉

            for i, d in enumerate(face_rects):                  # 取出所有偵測的結果
                x1, y1, x2, y2 = d.left(), d.top(), d.right(), d.bottom()

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4, cv2.LINE_AA) # 以方框標示偵測的人臉
                cv2.putText(frame, f'{scores[i]:.2f}, ({idx[i]:0.0f})', (x1, y1), font,    # 標示分數
                    0.7, (255, 255, 255), 1, lt)
                
            cv2.imshow('frame', frame) 
            out.write(frame)
        
    elif fms > 45500 and fms < 50000:
        d=5   
        img_D11 = cv2.GaussianBlur(frame, (d, d), 1.1)   
        img_D13 = cv2.GaussianBlur(frame, (d, d), 1.3)   

        img_D11_13 = img_D11 - img_D13   # try img_G0 + img_G1
        cv2.imshow('frame', img_D11_13)
        out.write(img_D11_13)

    elif fms > 50500 and fms < 55000:
        newHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        newHSV[:,:,2] = 230          # value 亮度 

        art = cv2.cvtColor(newHSV, cv2.COLOR_HSV2BGR)
        cv2.imshow('frame', art)
        out.write(art)
        
    elif fms > 55500 and fms < 60000:  #rotation
        
        rotation_speed = 0.5  # 調整旋轉的速度，數值越小速度越慢
        rotation_angle = (fms - 55500) * rotation_speed  # 計算旋轉的角度，乘以速度調整旋轉速度
        M1 = cv2.getRotationMatrix2D((w / 2, h / 2), rotation_angle, 1)  # 表示旋轉的中心點、旋轉的角度、圖像縮放因子
        rotate_img1 = cv2.warpAffine(frame, M1, (w, h))
        cv2.imshow('frame', rotate_img1)
        out.write(rotate_img1)

        
    else:
        cv2.imshow('frame', frame)
        out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1